In [1]:
import pandas as pd
df = pd.read_csv("train.csv")

In [2]:
import numpy as np
import os

## Чтение и nредварительная обработка текста 
text = ' '.join(list(df['response']))
start_indx = text.find('Шла вчера я по Садовой')
end_indx = text.find('Я, как мама, не люблю')

text = text[start_indx:end_indx]
char_set = set(text)
print("Oбщaя длина:", len(text))
print("Уникальных символов:", len(char_set))

Oбщaя длина: 98328
Уникальных символов: 92


In [3]:
chars_sorted = sorted(char_set)
char2int = {ch:i for i,ch in enumerate(chars_sorted)}
char_array = np.array(chars_sorted)

text_encoded = np.array(
    [char2int[ch] for ch in text],
    dtype=np.int32)

print('Paзмep закодированного текста: ', text_encoded.shape)

print(text[:18], '     == Кодирование ==> ', text_encoded[:15])
print(text_encoded[19:41], ' == Декодирование  ==> ', ''.join(char_array[text_encoded[19:41]]))

Paзмep закодированного текста:  (98328,)
Шла вчера я по Сад      == Кодирование ==>  [49 66 55  1 57 78 60 71 55  1 86  1 70 69  1]
[57 69 64  6  0 43 55 65  1 56 82 66 55  1 74 59 63 57 66 60 68 55]  == Декодирование  ==>  вой,
Так была удивлена


In [4]:
for ex in text_encoded[:12]:
    print('{} -> {}'.format(ex, char_array[ex]))

49 -> Ш
66 -> л
55 -> а
1 ->  
57 -> в
78 -> ч
60 -> е
71 -> р
55 -> а
1 ->  
86 -> я
1 ->  


In [5]:
seq_length = 46
chunk_size = seq_length+1

text_chunks = [text_encoded[i:i+chunk_size] 
               for i in range(len(text_encoded)-chunk_size+1)] 

## inspection:
for seq in text_chunks[:1]:
    input_seq = seq[:seq_length]
    target = seq[seq_length] 
    print(input_seq, ' -> ', target)
    print(repr(''.join(char_array[input_seq])), 
          ' -> ', repr(''.join(char_array[target])))

[49 66 55  1 57 78 60 71 55  1 86  1 70 69  1 42 55 59 69 57 69 64  6  0
 43 55 65  1 56 82 66 55  1 74 59 63 57 66 60 68 55  1 89  0 40 55]  ->  71
'Шла вчера я по Садовой,\nТак была удивлена —\nПа'  ->  'р'


In [6]:
import torch
from torch.utils.data import Dataset

class TextDataset(Dataset):
    def __init__(self, text_chunks):
        self.text_chunks = text_chunks

    def __len__(self):
        return len(self.text_chunks)
    
    def __getitem__(self, idx):
        text_chunk = self.text_chunks[idx]
        return text_chunk[:-1].long(), text_chunk[1:].long()
    
seq_dataset = TextDataset(torch.tensor(text_chunks))

C:\Users\Ainur\AppData\Local\Temp\ipykernel_12096\2527503007.py:15: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:281.)
  seq_dataset = TextDataset(torch.tensor(text_chunks))


In [7]:
for i, (seq, target) in enumerate(seq_dataset):
    print('Вход (х):', repr(''.join(char_array[seq])))
    print('Цель (у):', repr(''.join(char_array[target])))
    print()
    if i == 1:
        break
    

Вход (х): 'Шла вчера я по Садовой,\nТак была удивлена —\nПа'
Цель (у): 'ла вчера я по Садовой,\nТак была удивлена —\nПар'

Вход (х): 'ла вчера я по Садовой,\nТак была удивлена —\nПар'
Цель (у): 'а вчера я по Садовой,\nТак была удивлена —\nПаре'



In [8]:
device = torch.device("cuda:0")
from torch.utils.data import DataLoader
 
batch_size = 64

torch.manual_seed(1)
seq_dl = DataLoader(seq_dataset, batch_size=batch_size, shuffle=True, drop_last=True)

In [9]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, rnn_hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim) 
        self.rnn_hidden_size = rnn_hidden_size
        self.rnn = nn.LSTM(embed_dim, rnn_hidden_size, 
                           batch_first=True)
        self.fc = nn.Linear(rnn_hidden_size, vocab_size)

    def forward(self, x, hidden, cell):
        out = self.embedding(x).unsqueeze(1)
        out, (hidden, cell) = self.rnn(out, (hidden, cell))
        out = self.fc(out).reshape(out.size(0), -1)
        return out, hidden, cell

    def init_hidden(self, batch_size):
        hidden = torch.zeros(1, batch_size, self.rnn_hidden_size)
        cell = torch.zeros(1, batch_size, self.rnn_hidden_size)
        return hidden.to(device), cell.to(device)
    
vocab_size = len(char_array)
embed_dim = 256
rnn_hidden_size = 512

torch.manual_seed(1)
model = RNN(vocab_size, embed_dim, rnn_hidden_size) 
model = model.to(device)
model

RNN(
  (embedding): Embedding(92, 256)
  (rnn): LSTM(256, 512, batch_first=True)
  (fc): Linear(in_features=512, out_features=92, bias=True)
)

In [10]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

num_epochs = 2500

torch.manual_seed(1)

for epoch in range(num_epochs):
    hidden, cell = model.init_hidden(batch_size)
    seq_batch, target_batch = next(iter(seq_dl))
    seq_batch = seq_batch.to(device)
    target_batch = target_batch.to(device)
    optimizer.zero_grad()
    loss = 0
    for c in range(seq_length):
        pred, hidden, cell = model(seq_batch[:, c], hidden, cell) 
        loss += loss_fn(pred, target_batch[:, c])
    loss.backward()
    optimizer.step()
    loss = loss.item()/seq_length
    if epoch % 500 == 0:
        print(f'Эпоха {epoch} loss: {loss:.4f}')

Эпоха 0 loss: 4.5285
Эпоха 500 loss: 1.2750
Эпоха 1000 loss: 0.8007
Эпоха 1500 loss: 0.6771
Эпоха 2000 loss: 0.6386


In [11]:
from torch.distributions.categorical import Categorical

torch.manual_seed(1)

logits = torch.tensor([[1.0, 1.0, 1.0]])

print('Вероятности:', nn.functional.softmax(logits, dim=1).numpy()[0])

m = Categorical(logits=logits)
samples = m.sample((10,))
 
print(samples.numpy())

Вероятности: [0.33333334 0.33333334 0.33333334]
[[0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [2]
 [1]
 [1]]


In [12]:
torch.manual_seed(1)

logits = torch.tensor([[1.0, 1.0, 3.0]])

print('Вероятности:', nn.functional.softmax(logits, dim=1).numpy()[0])

m = Categorical(logits=logits)
samples = m.sample((10,))
 
print(samples.numpy())

Вероятности: [0.10650698 0.10650698 0.78698605]
[[0]
 [2]
 [2]
 [1]
 [2]
 [1]
 [2]
 [2]
 [2]
 [2]]


In [13]:
def sample(model, starting_str, 
           len_generated_text=500, 
           scale_factor=1.0):

    encoded_input = torch.tensor([char2int[s] for s in starting_str])
    encoded_input = torch.reshape(encoded_input, (1, -1))

    generated_str = starting_str

    model.eval()
    hidden, cell = model.init_hidden(1)
    hidden = hidden.to('cpu')
    cell = cell.to('cpu')
    for c in range(len(starting_str)-1):
        _, hidden, cell = model(encoded_input[:, c].view(1), hidden, cell) 
    
    last_char = encoded_input[:, -1]
    for i in range(len_generated_text):
        logits, hidden, cell = model(last_char.view(1), hidden, cell) 
        logits = torch.squeeze(logits, 0)
        scaled_logits = logits * scale_factor
        m = Categorical(logits=scaled_logits)
        last_char = m.sample()
        generated_str += str(char_array[last_char])
        
    return generated_str

torch.manual_seed(1)
model.to('cpu')
print(sample(model, starting_str='Шла вчера я'))

Шла вчера я не терпит
Чуждик Моют рожиц мне поражали!
С аспаху так поражевает,
Что волюке и мести очень улыбалась,
Чтоб в жизни не гряют в дневнике.
Он идем обливаться нету дождь
Над землёз пронется в чистый листом.
Что девочки и бородой,
В переводче своет доме.

От Андрюши:
Моря до летает
Догадаться,
Что же дедушка Морозу нужны,
Чтоб за то, что в школьной. Спит и ухал ростовый год! Есть в нарядные ение:
«Краша А елки
И однаждым огорченицо.

Все мечты осуществи! — Ну, тяте осталась
Приносита егоза сказка!



In [14]:
logits = torch.tensor([[1.0, 1.0, 3.0]])

print('Вероятности до масштабирования:', nn.functional.softmax(logits, dim=1).numpy()[0])

print("Вероятности nосле масштабирования 0.5:", nn.functional.softmax(0.5*logits, dim=1).numpy()[0])

print('Вероятности после масштабирования 0.1:', nn.functional.softmax(0.1*logits, dim=1).numpy()[0])

Вероятности до масштабирования: [0.10650698 0.10650698 0.78698605]
Вероятности nосле масштабирования 0.5: [0.21194156 0.21194156 0.57611686]
Вероятности после масштабирования 0.1: [0.3104238  0.3104238  0.37915248]


In [15]:
print(sample(model, starting_str='привет'))

приветство здесь смеется...
Рыпливо, Ты посмотрите, что спешим мы никакие отках,
В белые сели
По пояса, но полон!
Мы решили не молчат с вами заплавать пора в эту дочки. Борода, самой малышок! Ах, как корабли;
С клупки, поздравить мама:
– В мир знание – сила,
Во всегда мы много раз!
Но такое хвостика красный
Весёлым батарешку.
Любим её полянку,
А на уроках день-дарина,
Но такое и ни слова нет, чем мой юрный крик. Любимое смоетые,
Неважно, что растает.
Но в книжку у он их не узнались.
Слицы давай жить.



In [17]:
print(sample(model, starting_str='привет',scale_factor=0.5))

привета нету смеету,
И гориц Гармитикая, сплясался бор!
Но невётя мороз...
Вять лукворжевый! Рыжий?ка готовая.
— Ну, капилась мила
И совершиная
Сыгию, рона, нал он кто-то стоит,
Уставшие отличали:
- Шитаем!
В емУл Считомо,
Это пабли я кзо притки! Ручко – кроЖе,
Кругом стюнутый д
Ир жел у Кол,
А в боре снежной батаре.
Говорит:— оам
Матричница дюбочек.
Отехлилица нечагают
Буке мучи
И настроется вам надудачки, Капачке.
Но куда но пошёлте,
Где на вруды лепит
Долк
Луются волхода?
АМымите! 
Пусть немуров не


In [18]:
print(sample(model, starting_str='Вика',scale_factor=0.5))

Викала
Скачу На пути"?
Прогресть нахмуров и ренок,
Шерсть понимая мальчикку:
Любит напери-кряем.
Ведье жизнь, и кнов,
Ты тольное. Днес — Уколку.
Мы зажглась Зимой выша
Этая и ороно.

оучите Андрей: - Молоко, скорпевая
Пестей барщиw нь… Я эти мила -
От старим кто знаю.
Апешье себя привечае,
Когожка, Идет, ябребе белизнул,
Погая тоша дар-цазлопать,
Можно вам кудачной!—
Рiитётся общевия,
Прижное блеж…
А я двигда жалежим...
Эй, он держуз изблавы.
Все балогами — с наши влезщещь:
Могу, табаба я будет бязу
